# Método Simplex 

O método simplex é um procedimento inventado na metade final dos anos 1940 por George Dantzig para resolver programas lineares de forma iterativa. A cada iteração, dentro do espaço viável, a função objetivo é melhorada. Simultaneamente, ele resolve o problema dual, o que o torna um algoritmo robusto. A partir de um programa linear qualquer, o primeiro passo é transformar para a forma canônica e nesse formato, o problema é resolvido de maneira simples. 

## Forma canônica 

Um problema linear com a seguinte estrutura: 

1. As variáveis de decisão são não negativas; 
2. Todas as restrições são em forma de equações; 
3. Os coeficientes do lado direito das restrições são não negativos; 
4. Em cada restrição, uma variável de decisão é isolada com coeficiente +1, enquanto nas outras restrições, essa variável não aparece. Ela também aparece com coeficiente zero na função objetivo. 

## Critérios para solução do problema 

Quando o problema está na forma canônica, temos três importantes critérios: 

1. Critério da ilimitação: estabelece quando o problema é ilimitado ou não; 
2. Critério da melhora: estabelece quando e como melhorar a função objetivo a partir de uma variável não básica; 
3. Critério da razão e pivotamento: estabelece como alterar a base de variáveis.

In [1]:
import numpy as np
np.set_printoptions(precision=2)
from scipy.optimize import linprog

Estamos interessados em maximizar uma função objetivo linear sujeita a retrições lineares de igualdade e desigualdade. Dessa forma, resolvemos problemas do tipo: 

$$\begin{align}
\max_x \ & c^T x \\
\mbox{tal que}  \ & A_{ub} x \leq b_{ub}, \\
& A_{eq} x = b_{eq}, \\
& l \leq x \leq u
\end{align}$$

de forma que $x$ é o vetor de variáveis de decisão, $c$, $b_{ub}, b_{eq}, l, u$ são vetores e $A_{ub}$ e $A_{eq}$ são matrizes, todos escritos como `arrays` em `numpy`. 

## Desenvolvendo o algoritmo 

O primeiro passo, que pode ser deixado do lado, é conferir as dimensões do sistema.

In [2]:
def _check_dimensions(self, c, A_ub, A_eq, b_ub, b_eq): 

    if A_ub.shape[1] != self.n_var: 
        raise Exception("The number of columns of A_ub must be the number of lines of c.")
    elif A_eq.shape[1] != self.n_var:
        raise Exception("The number of columns of A_eq must be the number of lines of c.")
    elif b_ub.shape[0] != A_ub.shape[0]:
        raise Exception("The number of lines of A_ub must be the number of lines of b_ub.")
    elif b_eq.shape[0] != A_eq.shape[0]: 
        raise Exception("The number of lines of A_eq must be the number of lines of b_eq.")

A partir da inserção dos valores pelo usuário, precisamos ajeitar os valores de forma que eles fiquem no formato canônico, como descrito acima. Separamos o processo na seguintes etapas. 

1. Convertemos os limites superiores em desigualdades do problema, isto é, se $x \le u$, então adicionaremos essa restrição a matriz $A_ub$.  

Por padrão, assumimos que as variáveis são não negativas. Restrições do tipo $x_i \le u$ são inseridas como desigualdades normalmente. Restrições do tipo $x_i \ge l$ têm a substituição de variáveis $y_i = x_i - l \ge 0$. Assim, se $A_{ub} x \le b_{ub}$, teremos $A_{ub} \bar{x} \le b_{ub} + a^{i}_{ub}l$, tal que $\bar{x} = x$ exceto em $\bar{x}_i = y_i$ e $a^{i}_{ub}$ é a i-ésima coluna de $A_{ub}$.  Quando $l = -\infty$, escrevemos $x_i = x_i^+ - x_i^-$ em todas as restrições e na função objetivo, de forma que $x^+, x^- \ge 0$. Observo que estamos criando duas variáveis novas que depois serão juntas novamente para formar $x$. 

Todas essas transformações são armazenas para, posteriormente, voltarmos ao estado desejado pelo usuário. 

In [3]:
def _aux_free_variable(self, A_bar, A, col, i): 
    A_bar[:A.shape[0], col] = A[:,i]
    A_bar[:A.shape[0], col+1] = -A[:,i]
    return A_bar

def _bounds_handler(self, bounds, c, A_ub, A_eq, b_ub, b_eq):

    l = np.array([b[0] for b in bounds], dtype = np.float64)
    u = np.array([b[1] for b in bounds], dtype = np.float64)
    if len(bounds) == 0: 
        return (c, A_ub, A_eq, b_ub, b_eq)
    elif len(bounds) < self.n_var: 
        raise Exception("You need to specify dimension of c bounds.")
    else:
        free_variables = sum(l == -np.inf)
        upper_bounds = sum(u < np.inf)
        # Allocating space in case of free variables
        A_ub_bar = np.zeros((A_ub.shape[0] + upper_bounds, A_ub.shape[1] + free_variables))
        b_ub_bar = np.zeros((A_ub.shape[0] + upper_bounds, 1))
        b_ub_bar[:A_ub.shape[0],0:1] = b_ub
        A_eq_bar = np.zeros((A_eq.shape[0], A_eq.shape[1] + free_variables))
        c_bar = np.zeros((c.shape[0] + free_variables, 1))
        
        col = 0
        lin = 0
        for i in range(self.n_var):
            if l[i] == -np.inf: 
                A_ub_bar = self._aux_free_variable(A_ub_bar, A_ub, col, i)
                A_eq_bar = self._aux_free_variable(A_eq_bar, A_eq, col, i)
                c_bar = self._aux_free_variable(c_bar.transpose(), c.transpose(), col, i).transpose()
                # Save the necessary transformation after
                self.variables_transformations[i] = ([col, col+1], 0)
                if u[i] < np.inf: 
                    A_ub_bar[A_ub.shape[0]+lin,col] = 1
                    A_ub_bar[A_ub.shape[0]+lin,col+1] = -1
                    b_ub_bar[A_ub.shape[0]+lin] = u[i]
                    lin+=1
                col+=2
            else: 
                A_ub_bar[:A_ub.shape[0],col] = A_ub[:,i]
                A_eq_bar[:,col] = A_eq[:,i]
                c_bar[col] = c[i]
                
                self.variables_transformations[i] = ([col], l[i])
                if u[i] < np.inf: 
                    A_ub_bar[A_ub.shape[0]+lin,col] = 1
                    b_ub_bar[A_ub.shape[0]+lin] = u[i]
                    lin+=1
                b_ub_bar = b_ub_bar - A_ub_bar[:,col:col+1]*l[i]
                b_eq = b_eq - A_eq[:,i:i+1]*l[i]
                col+=1
    return (c_bar, A_ub_bar, A_eq_bar, b_ub_bar, b_eq)

2. Agora com as desigualdades em mãos, podemos adicionar as variáveis slack, quando $b$ é positivo na restrição $i$, e as variáveis surplus, quando $b$ é negativo. Na função, não diferenciamos esses tipos de variáveis. Com a adição de variáveis slack, teremos um sistema de equações como restrições do modelo que chamaremos de $Ax = b$. 

In [4]:
def _add_slack_variables(self, A_ub, b_ub):

    slack_var = np.eye(A_ub.shape[0])
    return (slack_var, A_ub, b_ub)

3. Lembre que queremos que o vetor $b$ apenas com entradas positivas. Por isso uma função para forçar isso!

In [5]:
def _force_b_positive(self, slack_var, A, b): 
    
    negative_const = np.where(b < 0)[0]
    A[negative_const] = -A[negative_const]
    slack_var[negative_const] = - slack_var[negative_const]
    b[negative_const] = -b[negative_const]
    
    return slack_var, A, b

4. Nem todas as restrições têm uma variável isolada, isto é, com coeficiente +1 e que não apareça nas outras. Por esse motivo, precisamos adicionar as variáveis artificiais. Para isso, vamos criar a base do sistema. Nessa base, para cada restrição (linha), procuramos uma variável (coluna) que tenha coeficiente +1 e seja a única de sua coluna com coeficiente não nulo. Além disso, essa variável não ṕode aparecer no vetor $c$. 

Abaixo, vemos a função que constrói uma base a priori das variáveis artificiais. 

In [17]:
def _assemble_basis(self, A, b, c): 
    
    self.basis = {i: None for i in range(A.shape[0])}
    for j in range(A.shape[1]): 
        test = 0.0 if j >= self.slack_pos else c[j]
        if test == 0: 
            non_zeros = np.where(A[:,j] != 0)[0]
            if (non_zeros.shape[0] == 1) & (A[non_zeros[0], j] > 0):
                self.basis[non_zeros[0]] = j
                A[non_zeros[0], :] = A[non_zeros[0],:]/A[non_zeros[0],j]
                b[non_zeros[0]] = b[non_zeros[0]]/A[non_zeros[0],j]
    return A, b

E depois, adicionamos as variáveis artificiais. 

In [13]:
def _add_artificial_variables(self, A, b, c): 
    
    art_var = 0
    self.art_constraints = []
    for i in self.basis.keys(): 
        if self.basis[i] is None: 
            self.basis[i] = A.shape[1] + art_var 
            self.art_constraints.append(i)
            art_var += 1
    artificial_variables = np.eye(art_var)

    self.table = np.zeros((A.shape[0] + 2, A.shape[1] + art_var + 1))
    self.table[:-2, :-art_var-1] = A
    self.table[self.art_constraints, -art_var-1:-1] = artificial_variables
    self.table[:-2, -1] = b.flatten()
    self.table[-2,:c.shape[0]] = c.flatten()

Com isso, teremos a forma canônica e estamos prontos para começar a otimização. A otimização ocorre segundo o procedimento fase 1 - fase 2. Nesse processo, fazemos uma iteração que procura uma variável para ser aumentada segundo o coeficiente na função objetivo e substituimos por outra na base. 

In [7]:
def _change_of_variables(self, tb, r, s): 
    
    a_rs = tb[r,s]
    norm_line_r = tb[r,:]/a_rs
    tb = tb - np.outer(tb[:,s], tb[r,:])/a_rs
    tb[r,:] = norm_line_r
    self.basis[r] = s
    
    return tb

def _iteration(self,tb, phase1=False): 

    while sum(tb[-1,:-1] > 0) > 0: 

        s = np.argmax(tb[-1,:-1])
        if sum(tb[:-1,s] > 0) == 0: 
            return None

        positive_a = np.where(tb[:-1-phase1,s] > 1e-16)[0]
        r = positive_a[(tb[positive_a,-1]/tb[positive_a,s]).argmin()]
        tb = self._change_of_variables(tb, r, s)
        
    return tb 

A fase 1 e 2 estão sintetizadas abaixo. O objetivo da fasse 1 é verifivar se as variáveis artificiais são nulas e, se sim, removê-las do sistema. Se não, o sistema é inviável. 

In [12]:
def _phase1(self, table): 

    new_objective = table[self.art_constraints, :].sum(axis=0)
    table[-1,:] = new_objective
    table[-1, self.art_pos:] = 0.0

    table = self._iteration(table, phase1 = True)
    
    if -table[-1,-1] < new_objective[-1]:
        return None
    else: 
        for (cons, basis) in self.basis.items(): 
            if basis >= self.art_pos: 
                for j in range(self.art_pos):
                    if table[cons, j] != 0: 
                        table = self._change_of_variables(table, cons, j)
                        break
        # keep the artificial variables always zero. 
        table = np.hstack([table[:,:self.art_pos], table[:,[-1]]])
        table = table[:-1,:]
    return table   

def _phase2(self, table):

    return self._iteration(table)

Vamos retornar, por fim, um objeto do tipo `OptimizeResult` que é descrito a seguir. Nesse caso `x` indica a solução ótima, `fun` o valor ótimo, `slack` os valores das variáveis slack e surplus, `sucess` se a otimização encerrou corretamente e `message` sobre o que aconteceu, caso algo tenha dado errado. 

In [8]:
class OptimizeResult: 
    
    def __init__(self, x, fun, slack, sucess, message): 
        
        self.x = x
        self.fun = fun
        self.slack = slack
        self.sucess = sucess
        self.message = message
        
        self.result = {"x": x, "fun": fun, "slack": slack, "success": sucess, "message": message}

A partir da tabela resultante do algoritmo, podemos calcular os resultados. 

In [9]:
def _results(self, tb): 

    x = np.zeros(self.n_var)
    y = np.zeros(self.slack_pos)
    slack = np.zeros(tb.shape[1]-1-self.slack_pos)
    for (cons, basis) in self.basis.items(): 
        if basis < self.slack_pos:
            y[basis] = tb[cons, -1] 
        else: 
            slack[basis-self.slack_pos] = tb[cons, -1]
    for i in range(self.n_var): 
        position = self.variables_transformations[i][0]
        if len(position) == 1:  
            x[i] = y[position[0]] + self.variables_transformations[i][1]
        else: 
            x[i] = y[position[0]] - y[position[1]] 

    fun = sum(self.c*x)
        
    return x, slack, fun

A função `optimize` é a chamada pelo usuário para realizar todo o procedimento. 

In [10]:
def optimize(self):

    tb = np.array(self.table)

    tb = self._phase1(tb)
    if tb is None: 
        message = "The problem is infeasible."
        sucess = False
        fun = None
        x = None
        slack = None
    else: 
        tb = self._phase2(tb)
        if tb is None: 
            message = "The primal problem is unbounded."
            sucess = False
            fun = None
            x = None
            slack = None
        else: 
            message = "Optimization terminated successfully."
            sucess = True
            x, slack, fun = self._results(tb)

    return OptimizeResult(x, fun, slack, sucess, message)

Por fim, definimos a classe para o método simplex. 

In [18]:
class SimplexMethod: 
    
    def __init__(self, c, **kwargs): 
        
        self.c = c
        c = c.reshape(-1,1)
        self.n_var = c.shape[0]
        A_ub = kwargs.get("A_ub", np.empty(shape=(0,self.n_var)))
        A_eq = kwargs.get("A_eq", np.empty(shape=(0,self.n_var)))
        b_ub = kwargs.get("b_ub", np.empty(shape=(0,1)))
        b_eq = kwargs.get("b_eq", np.empty(shape=(0,1)))
        # If bounds is an empty list, treating as [0, +inf].
        bounds = kwargs.get("bounds", [])
        
        b_ub = b_ub.reshape(-1,1)
        b_eq = b_eq.reshape(-1,1)
        
        self._check_dimensions(c, A_ub, A_eq, b_ub, b_eq)
        self.variables_transformations = {v: [] for v in range(self.n_var)}
        
        c, A_ub, A_eq, b_ub, b_eq = self._bounds_handler(bounds, c, A_ub, A_eq, b_ub, b_eq)
        slack, A_ub, b_ub = self._add_slack_variables(A_ub, b_ub)
        
        # Joining all the equalities
        slack = np.vstack([slack, np.zeros((A_eq.shape[0], slack.shape[0]))])
        A = np.vstack([A_ub, A_eq])
        b = np.vstack([b_ub, b_eq])
        slack, A, b = self._force_b_positive(slack, A, b)
        self.slack_pos = A.shape[1]
        A = np.hstack([A, slack])
        A, b = self._assemble_basis(A,b,c)
    
        self._add_artificial_variables(A, b, c)
        
        self.art_pos = A.shape[1]
        
        
SimplexMethod._check_dimensions = _check_dimensions
SimplexMethod._bounds_handler = _bounds_handler
SimplexMethod._aux_free_variable = _aux_free_variable
SimplexMethod._add_slack_variables = _add_slack_variables
SimplexMethod._force_b_positive = _force_b_positive
SimplexMethod._assemble_basis = _assemble_basis
SimplexMethod._add_artificial_variables = _add_artificial_variables
SimplexMethod._phase1 = _phase1
SimplexMethod._phase2 = _phase2
SimplexMethod._iteration = _iteration
SimplexMethod._change_of_variables = _change_of_variables
SimplexMethod.optimize = optimize
SimplexMethod._results = _results

## Exemplos

Considere o problema 

$$
\begin{align}
\max_x \ & -3x_1 + x_3 \\
\mbox{tal que }  &x_1 + x_2 + x_3 + x_4 \le 4, \\
& -2x_1 + x_2 - x_3 = 1 \\
& 3x_2 + x_3 + x_4 = 9, \\
& x_1 \ge -3, x_2 \le 4, x_j \ge 0, j = 2,3,4 
\end{align}
$$

In [19]:
c = np.array([-3, 0, 1, 0])
A_ub = np.array([[1,1,1,1]])
A_eq = np.array([[-2, 1, -1, 0], [0, 3,1,1]])
b_ub = np.array([4])
b_eq = np.array([1,9])
bounds = [[-3,np.inf], [0,4], [0, np.inf], [0, np.inf]]

In [20]:
model = SimplexMethod(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = bounds)
res = model.optimize()
res.result

{'x': array([-3.,  1.,  6.,  0.]),
 'fun': 15.0,
 'slack': array([0., 3.]),
 'success': True,
 'message': 'Optimization terminated successfully.'}

Podemos conferir com o otimizador do Scipy!

In [23]:
res2 = linprog(-c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = bounds, method = "simplex")
res2

     con: array([-2.22e-16,  0.00e+00])
     fun: -15.0
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([0.])
  status: 0
 success: True
       x: array([-3.,  1.,  6.,  0.])